In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
"""
INF6083 - Projet P1 : Tâche 0 - Échantillonnage Stratégique
BLOC 1 : Configuration et Imports
"""

import pandas as pd
import numpy as np
from datetime import datetime
import json
import gzip
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import os
import pickle
warnings.filterwarnings('ignore')

# Configuration
np.random.seed(42)
sns.set_style("whitegrid")

print("="*70)
print("ÉCHANTILLONNAGE STRATÉGIQUE - AMAZON BOOKS 2023")
print("="*70)

# ============================================================================
# CONFIGURATION DES PARAMÈTRES
# ============================================================================
REVIEWS_URL = "https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/review_categories/Books.jsonl"
METADATA_URL = "https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/resolve/main/raw/meta_categories/meta_Books.jsonl"

# Paramètres d'échantillonnage
MIN_REVIEWS_PER_USER = 20
TARGET_USERS = 50000
TARGET_REVIEWS_MIN = 500000
TARGET_REVIEWS_MAX = 2000000

# Paramètres temporels
START_YEAR = 2020
END_YEAR = 2023

# Fichiers de sauvegarde intermédiaires
CHECKPOINT_DIR = "checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("\n✅ Configuration chargée avec succès!")
print(f"   • MIN_REVIEWS_PER_USER: {MIN_REVIEWS_PER_USER}")
print(f"   • TARGET_USERS: {TARGET_USERS:,}")
print(f"   • PÉRIODE TEMPORELLE: {START_YEAR}-{END_YEAR}")
print(f"   • CHECKPOINTS: {CHECKPOINT_DIR}/")

In [ ]:
"""
BLOC 2 : Fonction de chargement et analyse de l'échantillon initial
"""

import requests

def load_jsonl_streaming(url, max_lines=None, show_progress=True):
    """Charge un fichier JSONL de manière efficace avec streaming"""
    print(f"\n📥 Chargement depuis: {url}")
    
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    data = []
    line_count = 0
    
    for line in tqdm(response.iter_lines(decode_unicode=True), desc="Lecture", disable=not show_progress):
        if not line:
            continue
        
        try:
            data.append(json.loads(line))
            line_count += 1
            
            if max_lines and line_count >= max_lines:
                break
                
        except json.JSONDecodeError:
            continue
    
    print(f"✅ {len(data):,} lignes chargées")
    return pd.DataFrame(data)

# ============================================================================
# CHARGEMENT ÉCHANTILLON INITIAL (skip si déjà fait)
# ============================================================================
SAMPLE_FILE = f"{CHECKPOINT_DIR}/sample_initial.pkl"

if os.path.exists(SAMPLE_FILE):
    print("\n♻️  Échantillon initial détecté - Chargement depuis le checkpoint...")
    sample_df = pd.read_pickle(SAMPLE_FILE)
    print(f"✅ {len(sample_df):,} lignes chargées depuis {SAMPLE_FILE}")
else:
    print("\n" + "="*70)
    print("PHASE 1: CHARGEMENT DE L'ÉCHANTILLON INITIAL")
    print("="*70)
    
    sample_df = load_jsonl_streaming(REVIEWS_URL, max_lines=100000)
    
    # Sauvegarder le checkpoint
    sample_df.to_pickle(SAMPLE_FILE)
    print(f"\n💾 Checkpoint sauvegardé: {SAMPLE_FILE}")

# Afficher les infos
print("\n📋 Structure des données:")
print(sample_df.head(3))
print(f"\n📏 Colonnes: {list(sample_df.columns)}")
print(f"📏 Lignes: {len(sample_df):,}")

# Vérifier format timestamp
sample_timestamp = sample_df['timestamp'].iloc[0]
print(f"\n🕐 Format timestamp:")
if sample_timestamp > 1e12:
    print(f"   • Format: millisecondes")
    print(f"   • Exemple converti: {datetime.fromtimestamp(sample_timestamp/1000).strftime('%Y-%m-%d')}")
else:
    print(f"   • Format: secondes")
    print(f"   • Exemple: {datetime.fromtimestamp(sample_timestamp).strftime('%Y-%m-%d')}")

# Analyse préliminaire
user_counts = sample_df['user_id'].value_counts()
print(f"\n📊 Statistiques (échantillon):")
print(f"   • Utilisateurs uniques: {len(user_counts):,}")
print(f"   • Moyenne reviews/user: {user_counts.mean():.2f}")
print(f"   • Médiane: {user_counts.median():.0f}")
print(f"   • Utilisateurs ≥{MIN_REVIEWS_PER_USER} reviews: {(user_counts >= MIN_REVIEWS_PER_USER).sum():,}")

print("\n✅ BLOC 2 TERMINÉ")

In [ ]:
"""
BLOC 3 : PASSE 1 - Comptage des évaluations par utilisateur
⏱️ Durée estimée: ~15 minutes
"""

USER_COUNTS_FILE = f"{CHECKPOINT_DIR}/user_counts.pkl"
SELECTED_USERS_FILE = f"{CHECKPOINT_DIR}/selected_users.pkl"

if os.path.exists(SELECTED_USERS_FILE):
    print("\n♻️  Utilisateurs déjà sélectionnés - Skip PASSE 1")
    with open(SELECTED_USERS_FILE, 'rb') as f:
        selected_users = pickle.load(f)
    print(f"✅ {len(selected_users):,} utilisateurs chargés depuis {SELECTED_USERS_FILE}")
    
else:
    print("\n" + "="*70)
    print("PHASE 3: PASSE 1 - COMPTAGE DES UTILISATEURS")
    print("="*70)
    print(f"\n🎯 Objectif: Identifier utilisateurs avec ≥{MIN_REVIEWS_PER_USER} évaluations")
    
    # Vérifier si le comptage existe déjà
    if os.path.exists(USER_COUNTS_FILE):
        print("\n♻️  Comptage existant détecté - Chargement...")
        with open(USER_COUNTS_FILE, 'rb') as f:
            user_review_counts = pickle.load(f)
        print(f"✅ {len(user_review_counts):,} utilisateurs chargés")
    else:
        print("\n📥 Comptage en cours (peut prendre ~15 minutes)...")
        user_review_counts = Counter()
        
        response = requests.get(REVIEWS_URL, stream=True)
        
        for line in tqdm(response.iter_lines(decode_unicode=True), desc="Comptage"):
            if not line:
                continue
            try:
                review = json.loads(line)
                user_review_counts[review['user_id']] += 1
            except (json.JSONDecodeError, KeyError):
                continue
        
        # Sauvegarder le comptage
        with open(USER_COUNTS_FILE, 'wb') as f:
            pickle.dump(user_review_counts, f)
        print(f"\n💾 Comptage sauvegardé: {USER_COUNTS_FILE}")
        print(f"✅ {len(user_review_counts):,} utilisateurs uniques")
    
    # Identifier utilisateurs actifs
    active_users = [user for user, count in user_review_counts.items() 
                    if count >= MIN_REVIEWS_PER_USER]
    
    print(f"\n📊 Utilisateurs actifs (≥{MIN_REVIEWS_PER_USER} reviews): {len(active_users):,}")
    
    # Échantillonner
    if len(active_users) > TARGET_USERS:
        selected_users = set(np.random.choice(active_users, size=TARGET_USERS, replace=False))
        print(f"✅ {TARGET_USERS:,} utilisateurs sélectionnés aléatoirement")
    else:
        selected_users = set(active_users)
        print(f"⚠️  {len(selected_users):,} utilisateurs disponibles (< {TARGET_USERS:,})")
    
    # Sauvegarder la sélection
    with open(SELECTED_USERS_FILE, 'wb') as f:
        pickle.dump(selected_users, f)
    print(f"💾 Sélection sauvegardée: {SELECTED_USERS_FILE}")

print(f"\n✅ BLOC 3 TERMINÉ - {len(selected_users):,} utilisateurs sélectionnés")

In [ ]:
"""
BLOC 4 : PASSE 2 - Extraction des évaluations des utilisateurs sélectionnés
⏱️ Durée estimée: ~12 minutes
"""

ACTIVE_SAMPLE_FILE = f"{CHECKPOINT_DIR}/df_active.pkl"

if os.path.exists(ACTIVE_SAMPLE_FILE):
    print("\n♻️  Échantillon 'Utilisateurs Actifs' déjà créé - Skip PASSE 2")
    df_active = pd.read_pickle(ACTIVE_SAMPLE_FILE)
    print(f"✅ {len(df_active):,} reviews chargées depuis {ACTIVE_SAMPLE_FILE}")
    
else:
    print("\n" + "="*70)
    print("PHASE 4: PASSE 2 - EXTRACTION DES ÉVALUATIONS")
    print("="*70)
    print(f"\n📥 Extraction des reviews de {len(selected_users):,} utilisateurs...")
    print("⏱️  Durée estimée: ~12 minutes")
    
    response = requests.get(REVIEWS_URL, stream=True)
    sampled_reviews_active = []
    
    for line in tqdm(response.iter_lines(decode_unicode=True), desc="Extraction"):
        if not line:
            continue
        try:
            review = json.loads(line)
            if review['user_id'] in selected_users:
                sampled_reviews_active.append(review)
        except (json.JSONDecodeError, KeyError):
            continue
    
    df_active = pd.DataFrame(sampled_reviews_active)
    
    # Convertir timestamps (millisecondes → secondes)
    print("\n🧹 Conversion des timestamps...")
    sample_ts = df_active['timestamp'].iloc[0]
    if sample_ts > 1e12:
        print("   ℹ️  Conversion: millisecondes → secondes")
        df_active['timestamp'] = df_active['timestamp'] / 1000
    
    # Filtrer timestamps invalides
    n_before = len(df_active)
    mask = (df_active['timestamp'] >= 946684800) & (df_active['timestamp'] <= 1893456000)
    df_active = df_active[mask].copy()
    n_after = len(df_active)
    
    if n_before != n_after:
        print(f"⚠️  {n_before - n_after:,} timestamps invalides supprimés ({((n_before-n_after)/n_before*100):.2f}%)")
    
    # Convertir en années
    df_active['year'] = pd.to_datetime(df_active['timestamp'], unit='s').dt.year
    
    # Sauvegarder
    df_active.to_pickle(ACTIVE_SAMPLE_FILE)
    print(f"\n💾 Échantillon sauvegardé: {ACTIVE_SAMPLE_FILE}")

# Afficher les statistiques
print(f"\n✅ ÉCHANTILLON 'UTILISATEURS ACTIFS':")
print(f"   • Reviews: {len(df_active):,}")
print(f"   • Utilisateurs: {df_active['user_id'].nunique():,}")
print(f"   • Livres: {df_active['parent_asin'].nunique():,}")
print(f"   • Période: {df_active['year'].min()} - {df_active['year'].max()}")
print(f"   • Rating moyen: {df_active['rating'].mean():.2f}")

print("\n✅ BLOC 4 TERMINÉ")

In [ ]:
"""
BLOC 5 : Échantillonnage temporel (2020-2023)
⏱️ Durée estimée: ~3 minutes
⚠️  OPTIONNEL - Peut être skippé si vous voulez seulement la stratégie 1
"""

TEMPORAL_SAMPLE_FILE = f"{CHECKPOINT_DIR}/df_temporal.pkl"

# Demander confirmation avant de lancer (car prend du temps)
RUN_TEMPORAL = True  # Mettre False pour skip

if not RUN_TEMPORAL:
    print("\n⏭️  BLOC 5 SKIPPÉ (échantillonnage temporel désactivé)")
    df_temporal = pd.DataFrame()  # DataFrame vide
    
elif os.path.exists(TEMPORAL_SAMPLE_FILE):
    print("\n♻️  Échantillon temporel déjà créé - Skip")
    df_temporal = pd.read_pickle(TEMPORAL_SAMPLE_FILE)
    print(f"✅ {len(df_temporal):,} reviews chargées depuis {TEMPORAL_SAMPLE_FILE}")
    
else:
    print("\n" + "="*70)
    print("PHASE 5: ÉCHANTILLONNAGE TEMPOREL")
    print("="*70)
    print(f"\n🎯 Objectif: Reviews de {START_YEAR} à {END_YEAR}")
    
    # Timestamps en millisecondes
    start_ts_ms = datetime(START_YEAR, 1, 1).timestamp() * 1000
    end_ts_ms = datetime(END_YEAR + 1, 1, 1).timestamp() * 1000
    
    print(f"\n📥 Filtrage temporel en cours...")
    
    response = requests.get(REVIEWS_URL, stream=True)
    sampled_reviews_temporal = []
    
    for line in tqdm(response.iter_lines(decode_unicode=True), desc="Filtrage"):
        if not line:
            continue
        try:
            review = json.loads(line)
            timestamp = review.get('timestamp', 0)
            if start_ts_ms <= timestamp < end_ts_ms:
                sampled_reviews_temporal.append(review)
        except (json.JSONDecodeError, KeyError):
            continue
    
    df_temporal = pd.DataFrame(sampled_reviews_temporal)
    
    if len(df_temporal) > 0:
        # Convertir timestamps
        df_temporal['timestamp'] = df_temporal['timestamp'] / 1000
        df_temporal['year'] = pd.to_datetime(df_temporal['timestamp'], unit='s').dt.year
        
        # Sauvegarder
        df_temporal.to_pickle(TEMPORAL_SAMPLE_FILE)
        print(f"\n💾 Échantillon temporel sauvegardé: {TEMPORAL_SAMPLE_FILE}")
    else:
        print("\n⚠️  Aucune donnée trouvée pour cette période")

# Afficher statistiques
if len(df_temporal) > 0:
    print(f"\n✅ ÉCHANTILLON TEMPOREL:")
    print(f"   • Reviews: {len(df_temporal):,}")
    print(f"   • Utilisateurs: {df_temporal['user_id'].nunique():,}")
    print(f"   • Livres: {df_temporal['parent_asin'].nunique():,}")
    print(f"   • Période: {df_temporal['year'].min()} - {df_temporal['year'].max()}")
else:
    print("\n⚠️  Pas d'échantillon temporel disponible")

print("\n✅ BLOC 5 TERMINÉ")

In [ ]:
"""
BLOC 6 : Comparaison des stratégies et recommandation
"""

print("\n" + "="*70)
print("PHASE 6: COMPARAISON ET RECOMMANDATION")
print("="*70)

# Tableau comparatif
comparison_data = {
    'Métrique': [
        'Nombre de reviews',
        'Nombre d\'utilisateurs',
        'Nombre de livres',
        'Reviews/utilisateur (moy)',
        'Reviews/livre (moy)',
        'Taux de sparsité (%)'
    ],
    'Utilisateurs Actifs': [
        f"{len(df_active):,}",
        f"{df_active['user_id'].nunique():,}",
        f"{df_active['parent_asin'].nunique():,}",
        f"{len(df_active) / df_active['user_id'].nunique():.2f}",
        f"{len(df_active) / df_active['parent_asin'].nunique():.2f}",
        f"{(1 - len(df_active) / (df_active['user_id'].nunique() * df_active['parent_asin'].nunique())) * 100:.4f}"
    ]
}

if len(df_temporal) > 0:
    comparison_data['Temporel'] = [
        f"{len(df_temporal):,}",
        f"{df_temporal['user_id'].nunique():,}",
        f"{df_temporal['parent_asin'].nunique():,}",
        f"{len(df_temporal) / df_temporal['user_id'].nunique():.2f}",
        f"{len(df_temporal) / df_temporal['parent_asin'].nunique():.2f}",
        f"{(1 - len(df_temporal) / (df_temporal['user_id'].nunique() * df_temporal['parent_asin'].nunique())) * 100:.4f}"
    ]
else:
    comparison_data['Temporel'] = ['N/A'] * 6

comparison_df = pd.DataFrame(comparison_data)
print("\n📊 TABLEAU COMPARATIF:")
print(comparison_df.to_string(index=False))

# Justification
print("\n📝 JUSTIFICATION DES STRATÉGIES:")
print("\n1️⃣ STRATÉGIE UTILISATEURS ACTIFS:")
print("   ✅ Avantages:")
print("      • Garantit des utilisateurs avec profils riches")
print("      • Réduit la sparsité de la matrice")
print("      • Facilite le calcul de similarités")
print("      • Volumétrie contrôlée et prévisible")
print("   ⚠️  Inconvénients:")
print("      • Biais vers les 'power users'")
print("      • Moins représentatif de l'utilisateur moyen")

print("\n2️⃣ STRATÉGIE TEMPORELLE:")
print("   ✅ Avantages:")
print("      • Données récentes et pertinentes")
print("      • Moins de biais de sélection")
print("      • Reflète les tendances actuelles")
print("   ⚠️  Inconvénients:")
print("      • Volumétrie variable selon la période")
print("      • Beaucoup d'utilisateurs occasionnels")

# Recommandation
active_in_range = TARGET_REVIEWS_MIN <= len(df_active) <= TARGET_REVIEWS_MAX
temporal_in_range = len(df_temporal) > 0 and TARGET_REVIEWS_MIN <= len(df_temporal) <= TARGET_REVIEWS_MAX

print("\n🎯 RECOMMANDATION:")
if active_in_range:
    recommended = "Utilisateurs Actifs"
    print(f"   ✅ Stratégie recommandée: {recommended}")
    print(f"   • Volumétrie dans la cible ({TARGET_REVIEWS_MIN:,} - {TARGET_REVIEWS_MAX:,})")
    final_df = df_active
    strategy_name = "active_users"
elif temporal_in_range:
    recommended = "Temporelle"
    print(f"   ✅ Stratégie recommandée: {recommended}")
    final_df = df_temporal
    strategy_name = "temporal"
else:
    print(f"   ⚠️  Aucune stratégie n'atteint exactement la cible")
    print(f"   • Recommandation par défaut: Utilisateurs Actifs")
    final_df = df_active
    strategy_name = "active_users"

print(f"\n✅ Stratégie finale: {strategy_name.upper()}")
print("\n✅ BLOC 6 TERMINÉ")

In [ ]:
"""
BLOC 7 : Sauvegarde des fichiers finaux (CSV + JSON)
"""

print("\n" + "="*70)
print("PHASE 7: SAUVEGARDE DES FICHIERS FINAUX")
print("="*70)

# Sauvegarder les échantillons CSV
print("\n💾 Sauvegarde des fichiers CSV...")

df_active.to_csv("amazon_books_sample_active_users.csv", index=False)
print(f"✅ amazon_books_sample_active_users.csv ({len(df_active):,} lignes)")

if len(df_temporal) > 0:
    df_temporal.to_csv("amazon_books_sample_temporal.csv", index=False)
    print(f"✅ amazon_books_sample_temporal.csv ({len(df_temporal):,} lignes)")

# Fichier recommandé
output_file = f"amazon_books_sample_{strategy_name}.csv"
final_df.to_csv(output_file, index=False)
print(f"✅ {output_file} (recommandé) ({len(final_df):,} lignes)")

# Métadonnées JSON
metadata = {
    'date_creation': datetime.now().isoformat(),
    'strategie_recommandee': strategy_name,
    'parametres': {
        'min_reviews_per_user': MIN_REVIEWS_PER_USER,
        'target_users': TARGET_USERS,
        'target_reviews_range': [TARGET_REVIEWS_MIN, TARGET_REVIEWS_MAX],
        'temporal_range': [START_YEAR, END_YEAR]
    },
    'resultats': {
        'active_users': {
            'n_reviews': len(df_active),
            'n_users': int(df_active['user_id'].nunique()),
            'n_items': int(df_active['parent_asin'].nunique()),
            'periode': f"{df_active['year'].min()}-{df_active['year'].max()}",
            'sparsity': float((1 - len(df_active) / (df_active['user_id'].nunique() * df_active['parent_asin'].nunique())) * 100)
        },
        'temporal': {
            'n_reviews': len(df_temporal) if len(df_temporal) > 0 else 0,
            'n_users': int(df_temporal['user_id'].nunique()) if len(df_temporal) > 0 else 0,
            'n_items': int(df_temporal['parent_asin'].nunique()) if len(df_temporal) > 0 else 0,
            'periode': f"{df_temporal['year'].min()}-{df_temporal['year'].max()}" if len(df_temporal) > 0 else "N/A"
        }
    }
}

with open('sampling_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
print("✅ sampling_metadata.json")

print("\n📁 FICHIERS GÉNÉRÉS:")
print("   • amazon_books_sample_active_users.csv")
if len(df_temporal) > 0:
    print("   • amazon_books_sample_temporal.csv")
print(f"   • {output_file} (recommandé)")
print("   • sampling_metadata.json")

print("\n✅ BLOC 7 TERMINÉ")

In [ ]:
"""
BLOC 8 : Visualisations et rapport final
"""

print("\n" + "="*70)
print("PHASE 8: VISUALISATIONS ET RAPPORT FINAL")
print("="*70)

# Créer les visualisations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribution des ratings
axes[0, 0].hist(final_df['rating'], bins=5, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 0].set_xlabel('Rating', fontsize=11)
axes[0, 0].set_ylabel('Fréquence', fontsize=11)
axes[0, 0].set_title('Distribution des Évaluations', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Reviews par utilisateur
user_review_dist = final_df['user_id'].value_counts()
axes[0, 1].hist(user_review_dist, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 1].set_xlabel('Nombre de reviews par utilisateur', fontsize=11)
axes[0, 1].set_ylabel('Fréquence', fontsize=11)
axes[0, 1].set_title('Distribution: Reviews par Utilisateur', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Reviews par livre (top 50)
book_review_dist = final_df['parent_asin'].value_counts().head(50)
axes[1, 0].bar(range(len(book_review_dist)), book_review_dist.values, alpha=0.7, color='seagreen')
axes[1, 0].set_xlabel('Livres (top 50)', fontsize=11)
axes[1, 0].set_ylabel('Nombre de reviews', fontsize=11)
axes[1, 0].set_title('Distribution: Reviews par Livre (Top 50)', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Distribution temporelle
year_dist = final_df['year'].value_counts().sort_index()
axes[1, 1].plot(year_dist.index, year_dist.values, marker='o', linewidth=2, 
                markersize=8, color='purple')
axes[1, 1].set_xlabel('Année', fontsize=11)
axes[1, 1].set_ylabel('Nombre de reviews', fontsize=11)
axes[1, 1].set_title('Distribution Temporelle', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('sampling_analysis.png', dpi=300, bbox_inches='tight')
print("\n✅ Visualisation sauvegardée: sampling_analysis.png")
plt.show()

# Rapport final
print("\n" + "="*70)
print("📊 RAPPORT FINAL")
print("="*70)

print(f"\n🎯 ÉCHANTILLON FINAL ({strategy_name.upper()}):")
print(f"   • Nombre de reviews: {len(final_df):,}")
print(f"   • Utilisateurs uniques: {final_df['user_id'].nunique():,}")
print(f"   • Livres uniques: {final_df['parent_asin'].nunique():,}")
print(f"   • Taux de sparsité: {(1 - len(final_df) / (final_df['user_id'].nunique() * final_df['parent_asin'].nunique())) * 100:.4f}%")
print(f"   • Rating moyen: {final_df['rating'].mean():.2f}")
print(f"   • Écart-type rating: {final_df['rating'].std():.2f}")
print(f"   • Période: {final_df['year'].min()} - {final_df['year'].max()}")

print("\n📈 MÉTRIQUES PAR UTILISATEUR:")
reviews_per_user = final_df['user_id'].value_counts()
print(f"   • Moyenne: {reviews_per_user.mean():.2f} reviews/utilisateur")
print(f"   • Médiane: {reviews_per_user.median():.0f}")
print(f"   • Min: {reviews_per_user.min()}")
print(f"   • Max: {reviews_per_user.max()}")

print("\n📚 MÉTRIQUES PAR LIVRE:")
reviews_per_book = final_df['parent_asin'].value_counts()
print(f"   • Moyenne: {reviews_per_book.mean():.2f} reviews/livre")
print(f"   • Médiane: {reviews_per_book.median():.0f}")
print(f"   • Min: {reviews_per_book.min()}")
print(f"   • Max: {reviews_per_book.max()}")

print("\n⭐ DISTRIBUTION DES RATINGS:")
for rating in sorted(final_df['rating'].unique()):
    count = (final_df['rating'] == rating).sum()
    pct = count / len(final_df) * 100
    print(f"   • {rating:.0f} étoiles: {count:,} ({pct:.1f}%)")

print("\n" + "="*70)
print("✅ ÉCHANTILLONNAGE STRATÉGIQUE TERMINÉ AVEC SUCCÈS!")
print("="*70)

print("\n📦 FICHIERS FINAUX GÉNÉRÉS:")
print("   1. amazon_books_sample_active_users.csv")
if len(df_temporal) > 0:
    print("   2. amazon_books_sample_temporal.csv")
print(f"   3. {output_file} (recommandé)")
print("   4. sampling_metadata.json")
print("   5. sampling_analysis.png")
print(f"\n📁 Checkpoints intermédiaires: {CHECKPOINT_DIR}/")

print("\n✅ BLOC 8 TERMINÉ - FIN DU SCRIPT")